<a href="https://colab.research.google.com/github/skywalker0803r/c620/blob/main/usage_example(colab)V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
!git clone https://github.com/skywalker0803r/c620.git
!pip install autorch > log.txt
!pip install optuna > log.txt

Cloning into 'c620'...
remote: Enumerating objects: 2205, done.
remote: Counting objects: 100% (1023/1023), done.
remote: Compressing objects: 100% (565/565), done.
remote: Total 2205 (delta 665), reused 687 (delta 449), pack-reused 1182
Receiving objects: 100% (2205/2205), 54.00 MiB | 21.28 MiB/s, done.
Resolving deltas: 100% (1443/1443), done.


# import

In [33]:
from c620.FV2 import AllSystem
from c620.configV2 import config
import joblib
import numpy as np
import pandas as pd
import os
os.chdir('/content/c620')
print(os.getcwd())
!git pull

/content/c620
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/skywalker0803r/c620
   a3876db..0a254a1  main       -> origin/main
Updating a3876db..0a254a1
Fast-forward
 usage_example(colab)V2.ipynb | 1606 ++++++++++++++++++++----------------------
 1 file changed, 752 insertions(+), 854 deletions(-)


# demo data

In [34]:
demo = joblib.load('./data/demo.pkl')
print(demo.keys())
icg_input = demo['icg_input']
c620_feed = demo['c620_feed']
t651_feed = demo['t651_feed']
icg_input.head()

dict_keys(['icg_input', 'c620_feed', 't651_feed', 'c620_op', 'c620_wt', 'c660_op', 'c660_wt', 'c670_op', 'c670_wt'])


,Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr,Simulation Case Conditions_Feed Rate_Feed from V615 Btm NA_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr,Simulation Case Conditions_Feed Rate_Feed from C820 Dist NA_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr,Simulation Case Conditions_Feed Rate_Feed from T651 NA_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Toluene_wt%,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw,Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr,Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw,Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC
002-002,163.0,1.356,10.321001,40.887009,15.0,2.338977,74.302254,17.467825,41.0,0.4524,44.244236,44.786339,70,980,0.25,10,36.0


In [35]:
icg_input[['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw']]

,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw
002-002,980


In [36]:
icg_input[['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%']]

,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%
002-002,70


In [37]:
icg_input[['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw']]

,Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw
002-002,10


# 1.試算模式

In [38]:
f = joblib.load('/content/c620/model/allsystem.pkl')

In [39]:
demo['icg_input']

,Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr,Simulation Case Conditions_Feed Rate_Feed from V615 Btm NA_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr,Simulation Case Conditions_Feed Rate_Feed from C820 Dist NA_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr,Simulation Case Conditions_Feed Rate_Feed from T651 NA_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Toluene_wt%,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw,Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr,Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw,Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC
002-002,163.0,1.356,10.321001,40.887009,15.0,2.338977,74.302254,17.467825,41.0,0.4524,44.244236,44.786339,70,980,0.25,10,36.0


In [40]:
c620_wt,c620_op,c660_wt,c660_op,c670_wt,c670_op = f.inference(demo['icg_input'],demo['c620_feed'],demo['t651_feed'])

# 檢查三項約束條件

In [41]:
c620_wt[['Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%']]

,Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%
002-002,71.350981


In [42]:
na_idx = [1,2,3,4,5,6,8,9,11,13,14,15,20,22,29] 
c660_wt.filter(regex='Side').filter(regex='wt%').iloc[:,na_idx].sum(axis=1).values[0]*10000

964.9365272378022

In [43]:
c660_wt['Benzene Column C660 Operation_Sidedraw (Benzene )Production Rate and Composition_Toluene_wt%'].values[0]*10000

9.149712539088709

In [44]:
c620_wt.iloc[:,:41].sum(axis=1)

002-002    100.0
dtype: float64

In [45]:
c660_wt.iloc[:,:41].sum(axis=1)

002-002    100.0
dtype: float64

In [46]:
c670_wt.iloc[:,:41].sum(axis=1)

002-002    100.0
dtype: float64

# 推薦模式

In [47]:
icg_input = demo['icg_input']
icg_input['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw'] -= 80
c620_wt2,c620_op2,c660_wt2,c660_op2,c670_wt2,c670_op2,bz_error,nainbz_error,tol_error = f.recommend(icg_input,demo['c620_feed'],demo['t651_feed'])

[I 2021-04-26 08:50:38,003] A new study created in memory with name: no-name-97abcf43-2b10-4ca4-81f5-62cc03841c86
[I 2021-04-26 08:50:38,081] Trial 0 finished with value: 6.624497219346784 and parameters: {'Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC': 174.89686937757665, 'Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC': 193.88407374106453, 'Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr': 4.149567679482194}. Best is trial 0 with value: 6.624497219346784.
[I 2021-04-26 08:50:38,158] Trial 1 finished with value: 0.5013953850490118 and parameters: {'Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC': 177.5207706376947, 'Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC': 194.66263406996768, 'Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr': 0.5002338075289394}. Best is trial 1 with value: 0.5013953850490118.


bz_error: 2.6904211551936896e-05
nainbz_error: 43.432289493090934
tol_error: 2.979267874181346e-06
dist_rate 1.0000124251363922
nainbz: 943.4322894930909


In [48]:
bz_error,nainbz_error,tol_error

(2.6904211551936896e-05, 43.432289493090934, 2.979267874181346e-06)

# 計算delta調幅

In [49]:
(c620_op2-c620_op).iloc[:,-2:]

,Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC,Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC
002-002,-0.863026,0.723152


In [50]:
(c660_op2-c660_op).iloc[:,-2:]

,Benzene Column C660 Operation_Column Temp Profile_C660 Tray 6 (SD & Control)_oC,Benzene Column C660 Operation_Column Temp Profile_C660 Tray 23 (Control)_oC
002-002,0.01978,-0.754673


In [51]:
(c670_op2-c670_op).iloc[:,-2:]

,Toluene Column C670 Operation_Column Temp Profile_C670 Tray 24 (Control)_oC,Toluene Column C670 Operation_Column Temp Profile_C670 Btm Temp (Control)_oC
002-002,-0.027344,0.071823


# 檢查三項約束條件

In [52]:
c620_wt2[['Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%']]

,Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%
002-002,69.999973


In [53]:
na_idx = [1,2,3,4,5,6,8,9,11,13,14,15,20,22,29] 
c660_wt2.filter(regex='Side').filter(regex='wt%').iloc[:,na_idx].sum(axis=1).values[0]*10000

943.4322894930909

In [54]:
c660_wt2['Benzene Column C660 Operation_Sidedraw (Benzene )Production Rate and Composition_Toluene_wt%'].values[0]*10000

10.000002979267874

In [55]:
c620_wt2.iloc[:,:41].sum(axis=1)

002-002    100.0
dtype: float64

In [56]:
c660_wt2.iloc[:,:41].sum(axis=1)

002-002    100.0
dtype: float64

In [57]:
c670_wt2.iloc[:,:41].sum(axis=1)

002-002    100.0
dtype: float64